# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)


{'avgTotalVolume': 56754663, 'calculationPrice': 'tops', 'change': 1.245, 'changePercent': 0.00699, 'close': None, 'closeSource': 'official', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': None, 'highSource': None, 'highTime': None, 'iexAskPrice': 179.45, 'iexAskSize': 400, 'iexBidPrice': 179.43, 'iexBidSize': 100, 'iexClose': 179.425, 'iexCloseTime': 1694456017543, 'iexLastUpdated': 1694456017543, 'iexMarketPercent': 0.016848390565728854, 'iexOpen': 180.08, 'iexOpenTime': 1694439000315, 'iexRealtimePrice': 179.425, 'iexRealtimeSize': 2, 'iexVolume': 653113, 'lastTradeTime': 1694456017543, 'latestPrice': 179.425, 'latestSource': 'IEX real time price', 'latestTime': '2:13:37 PM', 'latestUpdate': 1694456017543, 'latestVolume': None, 'low': None, 'lowSource': None, 'lowTime': None, 'marketCap': 2805172076600, 'o

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
price = data['latestPrice']
market_cap = data['marketCap']

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:
my_columns = ['Ticker','Stock Price', 'Market Capitalization','Number of Shares to Buy']
df = pd.DataFrame(columns= my_columns)
df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [7]:
df.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
    index = my_columns
    ),
    ignore_index=True
)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_20212\2999396054.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,179.425,2805172076600,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [8]:
df = pd.DataFrame(columns= my_columns)
for stock in stocks['Ticker'][:5]:
    api_url = f'https://cloud.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    df = df.append(
        pd.Series(
        [
            stock,
            data['latestPrice'],
            data['marketCap'],
            'N/A'
        ],
        index = my_columns
        ),
        ignore_index=True
     )

C:\Users\lenovo\AppData\Local\Temp\ipykernel_20212\116675288.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\lenovo\AppData\Local\Temp\ipykernel_20212\116675288.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\lenovo\AppData\Local\Temp\ipykernel_20212\116675288.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\lenovo\AppData\Local\Temp\ipykernel_20212\116675288.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\lenovo\AppData\Local\Temp\ipykernel_20212\116675288.py:5: FutureWarning: The frame.append method is deprecated and will

In [9]:
df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,113.420,33185248504,N/A
1,AAL,14.145,9241811657,N/A
2,AAP,62.920,3741868822,N/A
3,AAPL,179.425,2805172076600,N/A
4,ABBV,148.680,262427140382,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [10]:
def chunks(data, n):
    return [data[x:x+n] for x in range(0, len(data), n)]


In [11]:
symbol_groups = list(chunks(stocks['Ticker'],100))

df = pd.DataFrame(columns=my_columns)

# Fetch data for each chunk of symbols
for symbol_group in symbol_groups:
    symbol_string = ','.join(symbol_group)
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    
    response = requests.get(batch_api_call_url)
    
    if response.status_code == 200:
        data = response.json()
        rows_to_add = []
        for symbol in symbol_group:
            if symbol in data:
                row_data = [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['marketCap'],
                    'N/A'
                ]
                rows_to_add.append(row_data)

        if rows_to_add:
            chunk_df = pd.DataFrame(rows_to_add, columns=my_columns)
            df = pd.concat([df, chunk_df], ignore_index=True)
    else:
        print(f"Error fetching data for symbols {symbol_group}: Status Code {response.status_code}")

# Now you have a DataFrame containing the stock data
df


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,113.420,33185248504,N/A
1,AAL,14.145,9241811657,N/A
2,AAP,62.920,3741868822,N/A
3,AAPL,179.425,2805172076600,N/A
4,ABBV,148.680,262427140382,N/A
...,...,...,...,...
496,YUM,126.910,35561613672,N/A
497,ZBH,122.530,25604375339,N/A
498,ZBRA,256.855,13186515485,N/A
499,ZION,34.810,5156932080,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [16]:
while True:
    portfolio_size = input('Enter the value of your portfolio: ')
    
    try:
        val = float(portfolio_size)
        break  # Exit the loop if the conversion to float is successful
    except ValueError:
        print("That's not a valid number! Please try again.")


Enter the value of your portfolio: 10000000


In [23]:
position_size = val/len(df.index)
for i in range(0,len(df.index)):
    df.loc[i,'Number of Shares to Buy'] = math.floor(position_size/df.loc[i,'Stock Price'])

df   
# number_of_apple_shares = position_size/179.42
# print(math.floor(number_of_apple_shares))

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,113.420,33185248504,175
1,AAL,14.145,9241811657,1411
2,AAP,62.920,3741868822,317
3,AAPL,179.425,2805172076600,111
4,ABBV,148.680,262427140382,134
...,...,...,...,...
496,YUM,126.910,35561613672,157
497,ZBH,122.530,25604375339,162
498,ZBRA,256.855,13186515485,77
499,ZION,34.810,5156932080,573


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [40]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine = 'xlsxwriter')
df.to_excel(writer, 'Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [41]:
background_color = '#0a0a23'
font_color = '#ffffff'
string_format = writer.book.add_format(
    {
        'font_color' : font_color,
        'bg_color' : background_color,
        'border' : 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format' : '$0.00',
        'font_color' : font_color,
        'bg_color' : background_color,
        'border' : 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format' : '0',
        'font_color' : font_color,
        'bg_color' : background_color,
        'border' : 1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [42]:
# writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)
# writer.save()

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [43]:
column_formats = {
    'A': ['Ticker',string_format],
    'B': ['Stock Price',dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0],column_formats[column][1])


    

## Saving Our Excel Output

Saving our Excel file is very easy:

In [44]:
writer.save()